# Quadratic Programming
cvxopt 中的二次规划标准式：
$$
\begin{align}
\min_{x}\ &\frac{1}{2}x^TPx+q^Tx \\
subject\ to\ &Gx\preceq h \\
         &Ax=b
\end{align} 
$$

In [1]:
import numpy as np
import cvxopt as co
co.solvers.options['show_progress'] = False

In [2]:
def linear_hard_margin_SVM(X, y):
    """
    Linear Hard Margin SVM by QP
    Args:
        X: 数据
        y: 标签
    
    Returns:
        b: intercept
        w: 特征权重
    """
    P = np.block([
        [0.0,                                    np.zeros_like(X[0])],
        [np.zeros_like(X[0].reshape((-1,1))),    np.eye(len(X[0]))  ]
    ])
    q = np.zeros((len(X[0])+1,1), dtype=np.float64)
    G = -y.reshape((-1,1)) * np.hstack((np.ones((len(y), 1), dtype=np.float64), X))
    h = -1*np.ones_like(y, dtype=np.float64).reshape((-1,1))
    
    result =  np.array(co.solvers.qp(co.matrix(P), co.matrix(q), co.matrix(G), co.matrix(h))['x'])
    return result[0], result[1:]

In [3]:
X = np.array([[0,2,2,3], [0,2,0,0]]).T
y = np.array([-1,-1,1,1])
print(X, y)
print(-y.reshape((-1,1))*X)

[[0 0]
 [2 2]
 [2 0]
 [3 0]] [-1 -1  1  1]
[[ 0  0]
 [ 2  2]
 [-2  0]
 [-3  0]]


In [4]:
b, w = linear_hard_margin_SVM(X, y)
print('b = {}, \nw = {}'.format(b, w))

b = [-1.00000001], 
w = [[ 1.00000001]
 [-1.00000001]]


In [50]:
def linear_hard_margin_SVM_dual(X, y):
    """
    Linear Hard Margin SVM Dual by QP
    Args:
        X: 数据
        y: 标签
    
    Returns:
        alpha: 拉格朗日乘子系数
        sv_index: support vector index
        b: intercept
        w: 特征权重
    """
    P = y.reshape((-1,1))*(X.dot(X.T))*y.reshape((1,-1)).astype(np.float64)
    q = np.ones_like(y) * -1.0
    G = -1*np.eye(len(y),dtype=np.float64)
    h = np.zeros_like(y, dtype=np.float64).reshape((-1,1))
    A = y.reshape((1,-1)).astype(np.float64)
    b = 0.0
    
    alpha =  np.array(co.solvers.qp(co.matrix(P), co.matrix(q), co.matrix(G), co.matrix(h), co.matrix(A),co.matrix(b))['x']).reshape((-1,))
    sv_index = np.where(alpha>1e-6)[0]
    
    w = (alpha[sv_index]*y[sv_index]).dot(X[sv_index])
    n = np.random.choice(sv_index)
    b = y[n] - w.dot(X[n])
    return alpha, sv_index, w, b

In [52]:
alpha, _, w, b = linear_hard_margin_SVM_dual(X, y)
print('alpha: ', alpha)
print('w: ', w)
print('b: ', b)

alpha:  [5.00000000e-01 5.00000006e-01 9.99999998e-01 7.80987988e-09]
w:  [ 0.99999999 -1.00000001]
b:  -0.9999999478691657


In [53]:
X, y = np.array([[3,3],[4,3],[1,1]]), np.array([1,1,-1])   # 李航老师书的例子
alpha, _, w, b = linear_hard_margin_SVM_dual(X, y)
print('alpha: ', alpha)
print('w: ', w)
print('b: ', b)

alpha:  [2.49999916e-01 8.17639192e-08 2.49999998e-01]
w:  [0.49999975 0.49999975]
b:  -1.9999985061835712
